# Ethereum Fraud Detection

## Install Packages

In [ ]:
%pip install pandas numpy scikit-learn matplotlib seaborn

## Import Packages

In [ ]:
import pandas as pd

## Load Data

In [ ]:
df = pd.read_csv('dataset.csv')